In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
tqdm.pandas()
from IPython.display import display


# Begining of the project


In [2]:
reg_season=pd.read_csv('/Users/sebastiancoombs/Downloads/march-machine-learning-mania-2023/MRegularSeasonDetailedResults.csv')
reg_season

FileNotFoundError: [Errno 2] No such file or directory: '/Users/sebastiancoombs/Downloads/march-machine-learning-mania-2023/MRegularSeasonDetailedResults.csv'

In [ ]:
conf = pd.read_csv('/Users/sebastiancoombs/Downloads/march-machine-learning-mania-2023/MTeamConferences.csv')
conf['TeamID']=conf.Season.astype(str)+'_'+conf.TeamID.astype(str)
conf

,Season,TeamID,ConfAbbrev
0,1985,1985_1102,wac
1,1985,1985_1103,ovc
2,1985,1985_1104,sec
3,1985,1985_1106,swac
4,1985,1985_1108,swac
...,...,...,...
12657,2023,2023_1473,ovc
12658,2023,2023_1474,a_sun
12659,2023,2023_1475,ovc
12660,2023,2023_1476,nec


In [ ]:
reg_season[reg_season['WTeamID']==1246].Season.unique()

array([2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])

In [5]:

reg_season['WTeamID']=reg_season.Season.astype(str)+'_'+reg_season.WTeamID.astype(str)
reg_season['LTeamID']=reg_season.Season.astype(str)+'_'+reg_season.LTeamID.astype(str)
reg_season.head(3)
reg_season.Season.unique()

array([2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])

In [6]:
w_games=reg_season.filter(regex='^W',).copy()
w_games=pd.concat([reg_season['Season'],w_games],axis=1)
w_games.columns=[w.replace('W','',1) for w in w_games.columns]

l_games=reg_season.filter(regex='^L',).copy()
l_games=pd.concat([reg_season['Season'],l_games],axis=1)
l_games.columns=[l.replace('L','',1) for l in l_games.columns]


In [7]:
w_games

,Season,TeamID,Score,Loc,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF
0,2003,2003_1104,68,N,27,58,3,14,11,18,14,24,13,23,7,1,22
1,2003,2003_1272,70,N,26,62,8,20,10,19,15,28,16,13,4,4,18
2,2003,2003_1266,73,N,24,58,8,18,17,29,17,26,15,10,5,2,25
3,2003,2003_1296,56,N,18,38,3,9,17,31,6,19,11,12,14,2,18
4,2003,2003_1400,77,N,30,61,6,14,11,13,17,22,12,14,4,4,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107629,2023,2023_1104,82,N,27,66,12,36,16,22,21,29,16,12,6,7,19
107630,2023,2023_1272,75,A,27,53,6,12,15,21,9,27,11,11,4,2,18
107631,2023,2023_1343,74,H,23,57,9,23,19,24,11,25,11,5,3,2,18
107632,2023,2023_1345,67,N,24,62,6,28,13,20,13,25,17,7,5,0,14


In [8]:

#create season of games
games=pd.concat([w_games,l_games],axis=0)
games=games.sort_values(['TeamID'])



In [9]:
games=pd.merge(games,conf.drop('Season',axis=1),left_on='TeamID',right_on='TeamID',how='left')
games=games.set_index('TeamID')


In [10]:
games.loc['2009_1246'].Season.unique()

array([2009])

In [11]:
games.to_csv('Mens_Games_by_team.csv')


In [12]:
baseline=games.groupby(['ConfAbbrev','Season']).mean()
games=games.reset_index()

baseline.loc['sec',2009]

/var/folders/jc/lq6pqgbn3dlfs9v5g5x4jc_80000gn/T/ipykernel_87290/688823512.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  baseline=games.groupby(['ConfAbbrev','Season']).mean()


Score    73.856021
FGM      26.193717
FGA      58.753927
FGM3      6.706806
FGA3     19.426702
FTM      14.761780
FTA      21.447644
OR       12.424084
DR       25.026178
Ast      13.628272
TO       14.157068
Stl       7.311518
Blk       4.541885
PF       17.675393
Name: (sec, 2009), dtype: float64

In [13]:
#ignore ranking columns
def plus_minus(data):
    #pick out the team, season and conference 
    team=data.copy()
    teamid=team['TeamID'].unique()[0]
    season=int(team['TeamID'].unique()[0].split('_')[0])
    con=team.ConfAbbrev.unique()[0]
    team=team.drop(['ConfAbbrev','TeamID','Loc','Season'],axis=1)
    #locate the team baseline
    conf_avg=baseline.loc[con,season]
    #sumtract and sume for plus minus
    pm=team-conf_avg
    pm=pm.sum(axis=0)
    pm['ConfAbbrev']=con
    return pm

In [14]:
pm=games.groupby('TeamID').progress_apply(plus_minus)
pm=pm.reset_index()

pm.head(5)

100%|██████████| 7255/7255 [00:05<00:00, 1443.59it/s]


,TeamID,Score,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF,ConfAbbrev
0,2003_1102,-361.806867,-138.403433,-338.034335,44.991416,108.802575,-129.991416,-146.493562,-168.407725,-154.493562,-17.545064,-62.746781,-7.729614,-34.721030,-27.429185,mwc
1,2003_1103,200.221622,67.194595,39.272973,-21.567568,-44.410811,87.400000,91.959459,-36.064865,-65.486486,51.827027,-55.972973,13.421622,-10.337838,-19.251351,mac
2,2003_1104,-38.826211,-39.444444,33.478632,1.384615,52.159544,38.678063,38.205128,47.988604,12.358974,-67.837607,-30.609687,-24.561254,-2.410256,7.940171,sec
3,2003_1105,80.056338,-0.802817,65.894366,35.415493,47.190141,46.246479,36.739437,25.450704,-14.852113,36.795775,61.126761,39.767606,-20.521127,19.823944,swac
4,2003_1106,-142.323944,-27.633803,-106.267606,-3.014085,-36.718310,-84.042254,-111.126761,-6.591549,4.774648,-40.450704,20.521127,16.211268,7.746479,-36.112676,swac


In [15]:
pm.shape

(7255, 16)

In [16]:
pm[pm.TeamID=='2009_1246']

,TeamID,Score,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF,ConfAbbrev
2131,2009_1246,29.751309,-5.39267,-151.879581,-36.324607,-107.081152,76.861257,21.227749,-51.994764,44.136126,66.267016,106.816754,-16.280105,76.117801,4.712042,sec


In [17]:
seeds=pd.read_csv('/Users/sebastiancoombs/Downloads/march-machine-learning-mania-2023/MNCAATourneySeeds.csv')
seeds[seeds.TeamID==1246]


,Season,Seed,TeamID
27,1985,X12,1246
96,1986,Y01,1246
151,1987,X08,1246
225,1988,Y02,1246
449,1992,W02,1246
544,1993,Y01,1246
594,1994,X03,1246
688,1995,Z01,1246
720,1996,X01,1246
816,1997,Z01,1246


In [18]:
seeds['Team']=seeds['TeamID']
seeds['TeamID']=seeds.Season.astype(str)+'_'+seeds.TeamID.astype(str)
# seeds=seeds.drop('Season',axis=1)
seeds.head(3)

,Season,Seed,TeamID,Team
0,1985,W01,1985_1207,1207
1,1985,W02,1985_1210,1210
2,1985,W03,1985_1228,1228


In [19]:
def get_team(data):
  return int(data.split('_')[-1])
  
def get_season(data):
  return int(data.split('_')[-0])

def fill_seed(data):
  data=data.fillna(np.random.randint(1,17))
  return data

In [20]:
pm['Team']=pm['TeamID'].apply(get_team)
pm['Season']=pm['TeamID'].apply(get_season)
pm.shape

(7255, 18)

In [21]:
len(list(seeds.Team.unique())),len(list(pm.Team.unique()))

(306, 368)

In [22]:
pm_seed=pd.merge(pm,seeds,left_on=['Team','Season'],right_on=['Team','Season'], how='left')

pm_seed['Seed']=pd.to_numeric(pm_seed['Seed'].str.replace("[a-zA-Z]","",regex=True),errors='coerce')
pm_seed['Seed']=pm_seed['Seed'].apply(lambda l: l if not np.isnan(l) else np.random.choice(range(1,17)))
pm_seed=pm_seed.drop('TeamID_y',axis=1)

pm_seed

,TeamID_x,Score,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF,ConfAbbrev,Team,Season,Seed
0,2003_1102,-361.806867,-138.403433,-338.034335,44.991416,108.802575,-129.991416,-146.493562,-168.407725,-154.493562,-17.545064,-62.746781,-7.729614,-34.721030,-27.429185,mwc,1102,2003,16.0
1,2003_1103,200.221622,67.194595,39.272973,-21.567568,-44.410811,87.400000,91.959459,-36.064865,-65.486486,51.827027,-55.972973,13.421622,-10.337838,-19.251351,mac,1103,2003,6.0
2,2003_1104,-38.826211,-39.444444,33.478632,1.384615,52.159544,38.678063,38.205128,47.988604,12.358974,-67.837607,-30.609687,-24.561254,-2.410256,7.940171,sec,1104,2003,10.0
3,2003_1105,80.056338,-0.802817,65.894366,35.415493,47.190141,46.246479,36.739437,25.450704,-14.852113,36.795775,61.126761,39.767606,-20.521127,19.823944,swac,1105,2003,1.0
4,2003_1106,-142.323944,-27.633803,-106.267606,-3.014085,-36.718310,-84.042254,-111.126761,-6.591549,4.774648,-40.450704,20.521127,16.211268,7.746479,-36.112676,swac,1106,2003,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7250,2023_1473,-147.517007,-36.809524,-46.588435,-4.091837,-41.969388,-69.806122,-113.860544,-43.568027,-22.333333,-50.476190,18.503401,-16.612245,9.632653,-60.401361,ovc,1473,2023,15.0
7251,2023_1474,147.978102,9.197080,38.364964,20.781022,60.605839,108.802920,151.445255,46.832117,40.072993,8.496350,18.021898,3.839416,-16.233577,5.605839,a_sun,1474,2023,14.0
7252,2023_1475,90.482993,20.190476,73.411565,58.908163,124.030612,-8.806122,10.139456,58.431973,33.666667,-18.476190,5.503401,5.387755,-23.367347,4.598639,ovc,1475,2023,10.0
7253,2023_1476,-18.835821,-36.358209,-128.716418,22.268657,37.582090,31.611940,5.291045,-91.895522,21.597015,-24.268657,6.761194,-1.335821,-6.119403,-34.843284,nec,1476,2023,10.0


In [23]:

# pm_seed.to_csv('NCAA_teams_plus_minus.csv')

In [24]:
pm_seed.shape

(7255, 19)

In [25]:
pm_seed[pm_seed.Seed.notna()]

,TeamID_x,Score,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF,ConfAbbrev,Team,Season,Seed
0,2003_1102,-361.806867,-138.403433,-338.034335,44.991416,108.802575,-129.991416,-146.493562,-168.407725,-154.493562,-17.545064,-62.746781,-7.729614,-34.721030,-27.429185,mwc,1102,2003,16.0
1,2003_1103,200.221622,67.194595,39.272973,-21.567568,-44.410811,87.400000,91.959459,-36.064865,-65.486486,51.827027,-55.972973,13.421622,-10.337838,-19.251351,mac,1103,2003,6.0
2,2003_1104,-38.826211,-39.444444,33.478632,1.384615,52.159544,38.678063,38.205128,47.988604,12.358974,-67.837607,-30.609687,-24.561254,-2.410256,7.940171,sec,1104,2003,10.0
3,2003_1105,80.056338,-0.802817,65.894366,35.415493,47.190141,46.246479,36.739437,25.450704,-14.852113,36.795775,61.126761,39.767606,-20.521127,19.823944,swac,1105,2003,1.0
4,2003_1106,-142.323944,-27.633803,-106.267606,-3.014085,-36.718310,-84.042254,-111.126761,-6.591549,4.774648,-40.450704,20.521127,16.211268,7.746479,-36.112676,swac,1106,2003,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7250,2023_1473,-147.517007,-36.809524,-46.588435,-4.091837,-41.969388,-69.806122,-113.860544,-43.568027,-22.333333,-50.476190,18.503401,-16.612245,9.632653,-60.401361,ovc,1473,2023,15.0
7251,2023_1474,147.978102,9.197080,38.364964,20.781022,60.605839,108.802920,151.445255,46.832117,40.072993,8.496350,18.021898,3.839416,-16.233577,5.605839,a_sun,1474,2023,14.0
7252,2023_1475,90.482993,20.190476,73.411565,58.908163,124.030612,-8.806122,10.139456,58.431973,33.666667,-18.476190,5.503401,5.387755,-23.367347,4.598639,ovc,1475,2023,10.0
7253,2023_1476,-18.835821,-36.358209,-128.716418,22.268657,37.582090,31.611940,5.291045,-91.895522,21.597015,-24.268657,6.761194,-1.335821,-6.119403,-34.843284,nec,1476,2023,10.0


In [26]:
pm_seed.shape

(7255, 19)

In [27]:
pm_seed[pm_seed.Seed.notna()]

,TeamID_x,Score,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF,ConfAbbrev,Team,Season,Seed
0,2003_1102,-361.806867,-138.403433,-338.034335,44.991416,108.802575,-129.991416,-146.493562,-168.407725,-154.493562,-17.545064,-62.746781,-7.729614,-34.721030,-27.429185,mwc,1102,2003,16.0
1,2003_1103,200.221622,67.194595,39.272973,-21.567568,-44.410811,87.400000,91.959459,-36.064865,-65.486486,51.827027,-55.972973,13.421622,-10.337838,-19.251351,mac,1103,2003,6.0
2,2003_1104,-38.826211,-39.444444,33.478632,1.384615,52.159544,38.678063,38.205128,47.988604,12.358974,-67.837607,-30.609687,-24.561254,-2.410256,7.940171,sec,1104,2003,10.0
3,2003_1105,80.056338,-0.802817,65.894366,35.415493,47.190141,46.246479,36.739437,25.450704,-14.852113,36.795775,61.126761,39.767606,-20.521127,19.823944,swac,1105,2003,1.0
4,2003_1106,-142.323944,-27.633803,-106.267606,-3.014085,-36.718310,-84.042254,-111.126761,-6.591549,4.774648,-40.450704,20.521127,16.211268,7.746479,-36.112676,swac,1106,2003,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7250,2023_1473,-147.517007,-36.809524,-46.588435,-4.091837,-41.969388,-69.806122,-113.860544,-43.568027,-22.333333,-50.476190,18.503401,-16.612245,9.632653,-60.401361,ovc,1473,2023,15.0
7251,2023_1474,147.978102,9.197080,38.364964,20.781022,60.605839,108.802920,151.445255,46.832117,40.072993,8.496350,18.021898,3.839416,-16.233577,5.605839,a_sun,1474,2023,14.0
7252,2023_1475,90.482993,20.190476,73.411565,58.908163,124.030612,-8.806122,10.139456,58.431973,33.666667,-18.476190,5.503401,5.387755,-23.367347,4.598639,ovc,1475,2023,10.0
7253,2023_1476,-18.835821,-36.358209,-128.716418,22.268657,37.582090,31.611940,5.291045,-91.895522,21.597015,-24.268657,6.761194,-1.335821,-6.119403,-34.843284,nec,1476,2023,10.0


In [28]:
teams=pd.read_csv('/Users/sebastiancoombs/Downloads/march-machine-learning-mania-2023/MTeams.csv')
teams[teams.TeamName=='Kentucky'].TeamID.unique()

array([1246])

In [29]:
data1=pd.merge(teams[['TeamID','TeamName']],pm_seed,left_on='TeamID',right_on='Team',how='left',)
data1['TeamID']=data1['TeamID_x']
data1=data1.drop(['TeamID_x'],axis=1)
first=list(data1.filter(like='Team').columns)
second=[c for c in data1.columns if c not in first]
order=first+second
data1=data1[order]
data1.columns

Index(['TeamID', 'TeamName', 'Team', 'Score', 'FGM', 'FGA', 'FGM3', 'FGA3',
       'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'ConfAbbrev',
       'Season', 'Seed'],
      dtype='object')

In [30]:
data1.to_csv('Mens_pm_w_names.csv',index=False)